In [ ]:
0) SHIFT START (5–10 min)

✅ Check handover notes (Teams/Email)
✅ Check open tickets + priority + SLA
✅ Any planned changes / maintenance today?
✅ Note critical alerts / data gaps reported

🟩 1) SEARCH HEAD HEALTH (SH)
✅ What to check:

📍 Monitoring Console → Search → Instance
✔ Concurrency
✔ Search latency
✔ Long running searches

📌 Meaning:

Concurrency = how many searches running same time

Latency = waiting time before search starts

Long running = searches taking too long to finish

🔎 SPL (SH errors)
index=_internal sourcetype=splunkd ERROR host=<search_head_host>

🛠 T/S

✅ Too many searches → stop heavy searches / inform users
✅ Check CPU/MEM (SH)
✅ Check scheduler backlog
✅ Identify long searches:

index=_internal sourcetype=splunkd "search finished"
| stats max(total_run_time) as maxTime avg(total_run_time) as avgTime by user savedsearch_name
| sort -maxTime


✅ Suggest optimization (add index filter, reduce time range, avoid join/transaction)

🟧 2) INDEXER HEALTH (IDX)
✅ What to check:

📍 Monitoring Console → Indexing → Performance
✔ indexing rate
✔ throughput
✔ CPU/memory

📍 Resource Usage → Disk
✔ hot/warm disk < 80%

🔎 SPL (Indexer errors)
index=_internal sourcetype=splunkd ERROR

🛠 T/S

✅ Disk high → alert infra + increase space / freeze policy check
✅ Indexing drop → check forwarders / ingestion pipelines
✅ Queue blocking check:

index=_internal sourcetype=splunkd group=queue
| stats max(current_size) as max_size by name
| sort - max_size

🟨 3) HEAVY FORWARDER HEALTH (HF)
✅ What to check:

Login: http://<heavy_forwarder_host>:8000
📍 Settings → Forwarder Monitoring
✔ no warnings
✔ forwarding stable

🔎 SPL (HF errors)
index=_internal sourcetype=splunkd ERROR host=<heavy_forwarder_host>

🛠 T/S

✅ Check output connectivity:

index=_internal sourcetype=splunkd component=TcpOutputProc (ERROR OR WARN)


✅ If blocked queue → restart inputs (if allowed)
✅ Validate network connectivity to indexers (9997)

🟥 4) LICENSE MANAGER CHECK (DAILY MUST)
✅ What to check:

📍 Settings → Licensing
✔ any violations?
✔ usage spike?

🔎 SPL (License usage)
index=_internal source=*license_usage.log

🛠 T/S

✅ Find top license consuming indexes:

index=_internal source=*license_usage.log
| stats sum(b) as bytes by idx
| eval GB=round(bytes/1024/1024/1024,2)
| sort -GB


✅ If spike → identify sourcetype + source
✅ Stop noisy logs / apply filtering

🟦 5) DATA INGESTION CHECK (MOST IMPORTANT)
✅ What to check:

✔ last 15 min ingestion count

index=* earliest=-15m
| stats count by index sourcetype
| sort -count


✔ detect ingestion gap

| tstats latest(_time) as lastTime where index=* by index sourcetype
| eval gap=now()-lastTime
| where gap>900
| eval gap_minutes=round(gap/60,2)
| table index sourcetype gap_minutes lastTime

🛠 T/S

✅ If count = 0 / gap found → create Data Gap ticket
✅ Validate forwarder service / port / inputs.conf
✅ Confirm connectivity to indexers

🟩 6) SHC (SEARCH HEAD CLUSTER) CHECK
✅ Terms:

✔ Captain Status = leader node
✔ Replication Status = config/app replication
✔ Member Sync Issue = one SH not updated

🛠 T/S (CLI on SH)
$SPLUNK_HOME/bin/splunk show shcluster-status


Check bundle sync:

$SPLUNK_HOME/bin/splunk show shcluster-bundle-status


✅ If member out-of-sync → check network / disk / captain stability

🟪 7) DEPLOYER (SHC DEPLOYMENT) – CLEAN BUNDLE CHECK
✅ How to check bundle is clean:

🔎 Deployer log check:

index=_internal sourcetype=splunkd host=<deployer_host> (bundle OR deploy OR shcluster) (ERROR OR WARN OR FAIL)
| table _time host log_level component message
| sort - _time

✅ How to confirm no deployment failure:

On deployer:

$SPLUNK_HOME/bin/splunk show shcluster-bundle-status

🛠 T/S

✅ If bundle fails:

check apps permissions

fix invalid .conf stanza

remove unnecessary large apps

re-push bundle

Bundle push:

$SPLUNK_HOME/bin/splunk apply shcluster-bundle -target https://<captain_ip>:8089 -auth admin:password

🟫 8) DEPLOYMENT SERVER CHECK (FORWARDERS)
✅ What to check:

Login: http://<deployment_server_host>:8000
📍 Forwarder Management
✔ clients connected
✔ last phone home time ok
✔ server classes ok

🛠 T/S

✅ If forwarder missing:

validate deploymentclient.conf

check firewall / connectivity

restart splunkforwarder service

🟥 9) SCHEDULER / ALERT HEALTH (ES SOC)
✅ Check skipped/failed searches:
index=_internal sourcetype=scheduler status=skipped OR status=failed
| stats count by savedsearch_name status
| sort -count

🛠 T/S

✅ Increase resources / adjust schedule windows
✅ Reduce concurrency conflicts
✅ Optimize correlation searches

🟦 10) END OF SHIFT HANDOVER (Mandatory)

✅ Closed tickets list
✅ Pending tickets + next action
✅ Data gaps + impacted source
✅ Risk items: disk/license/concurrency
✅ Escalations to L2/L3